In [11]:
from keras_vggface.utils import preprocess_input
from project.dg_face import utils, detection, alignment, recognition
import os
import numpy as np

## Как будем строить датасет.
* 50 различных сотрудников в компании (можно начать с 10 для картиночки)
* у каждого сотрудника по 3-5 фотографий в тренировочном датасете
* у каждого сорудника по 1(сомнительно) фотографии в тестовом датасете

### Вне компании
* 10 человек вне компании
* их фотографий в тренировочном датасете нет
* они в тестовом датасете


In [31]:
model = recognition.make_cnn_model()
def get_embeddings(base_path):
    X, y = [], []
    for person in os.listdir(base_path):
        for person_img in os.listdir(base_path + "/" + person):
            path = base_path + '/' + person + '/' + person_img
            print(path)
            img = utils.load_image(path)
            faces = detection.detect_faces(img)
            if len(faces) != 1:
                print(path, 'was skipped because of len(faces)=', len(faces))
                continue
            face_image = alignment.aligned_face(img, faces[0])
            face_image = preprocess_input(face_image.astype(float), version=1)
            encoding = model.predict(np.array([face_image]))[0]
            X.append(encoding)
            y.append(person)
    return X, y

In [32]:
X_train, y_train = get_embeddings("images_30_20/train")
X_test, y_test = get_embeddings("images_30_20/test")
print(len(X_train), len(X_test))

images_30_20/train/worker0/001257.jpg
1/1 [==============================] - 0s 125ms/step
images_30_20/train/worker0/008392.jpg
1/1 [==============================] - 0s 20ms/step
images_30_20/train/worker0/013825.jpg
1/1 [==============================] - 0s 19ms/step
images_30_20/train/worker1/185949.jpg
1/1 [==============================] - 0s 26ms/step
images_30_20/train/worker1/187455.jpg
1/1 [==============================] - 0s 24ms/step
images_30_20/train/worker1/194128.jpg
1/1 [==============================] - 0s 20ms/step
images_30_20/train/worker10/165300.jpg
1/1 [==============================] - 0s 20ms/step
images_30_20/train/worker10/168172.jpg
1/1 [==============================] - 0s 24ms/step
images_30_20/train/worker10/173174.jpg
1/1 [==============================] - 0s 20ms/step
images_30_20/train/worker11/186581.jpg
1/1 [==============================] - 0s 19ms/step
images_30_20/train/worker11/191573.jpg
1/1 [==============================] - 0s 20ms/step
imag

In [22]:
X_train[0][:5]
# X_test[0][:5]

array([ 1.7187077,  6.0274   , 11.638269 ,  8.601409 ,  1.8861231],
      dtype=float32)

In [33]:
np.savez_compressed("npz_30_20/train", X=np.array(X_train), y=np.array(y_train))
np.savez_compressed("npz_30_20/test", X=np.array(X_test), y=np.array(y_test))